### Charging Profile

In [79]:
from scipy.optimize import fsolve
import numpy as np


def charge_profile(
    soc: float,
    d_c: float,
    cap: float = 70,
    ev_mpi: float = 7.36,
    k: float = 0.1,
) -> list:
    """
    Calculate the charging profile for a given EV.

    Parameters
    ----------
    soc : float
        State of charge of the battery in %.
    d_c : float
        Desired charge of the battery in %.
    cap : float
        Capacity of the battery in kWh.
    ev_mpi : float
        Maximum power input of the EV in kW.
    k : float
        Charging curve constant.
        0.01-0.03 charge aggressively,
        0.05-0.1  prioritizing battery health and longevity
    cv : float
        Capacity at which the charging curve flattens in %.
    cc : float
        Capacity at which the charging curve starts in %.

    Returns
    -------
    list
        List of the charging profile in the format [time, power, charge].
    """
    # --------------------------------------------------------------------------
    # Calculate the charging profile
    # --------------------------------------------------------------------------
    # Calculate the charge required to reach the desired capacity
    cc = soc * cap
    dc = d_c * cap
    cv = 0.8 * cap

    # first part of charge < 80% of cap
    c1 = min(dc, cv) - cc  #          charge [kWh] required
    p1 = ev_mpi if c1 > 0 else 0  #   power [kW] required
    t1 = 0 if c1 == 0 else c1 / p1  # time [h] required

    # second part of charge > 80% of cap
    c2 = max(dc - cv, 0)  #           second part of charge > 80% of cap
    p2 = 0  #                          initial power [kW] required
    t2 = 0  #                          initial time [h] required

    # Define the function for the given equation
    def equation(t):
        return ev_mpi * (t - t1) * np.exp(-k * (t - t1)) - c2

    # Initial guess
    initial_guess = t1

    # dtermime time and power for 2nd part of charge CV
    if c2 > 0:
        # Solve the equation numerically
        # https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.fsolve.html
        solution = fsolve(func=equation, x0=initial_guess, xtol=1e-3)

        t2 = solution[0]  #    time [h] required 2nd part CV
        p2 = c2 / (t2 - t1)  # power [kW] required 2nd part CV

    e2 = 0 if t2 == 0 else float(equation(t2))

    return {
        "charge": {"cc": cc, "c1": c1, "c2": c2},
        "result": {"fc": cc + c1 + c2 + e2, "dc": dc, "rm": dc - (cc + c1 + c2 + e2)},
        "chrg_1": {"t": t1, "p": p1},
        "chrg_2": {"t": t2, "p": p2},
    }

In [80]:
# --------------------------------------------------------------------------
ev_mpi = 7.36
soc = 0.47
d_c = 1
cap = 70

k = 0.1

print("\nFunction results\n")
print(charge_profile(soc, d_c, cap, ev_mpi, k))


Function results

{'charge': {'cc': 32.9, 'c1': 23.1, 'c2': 14.0}, 'result': {'fc': 69.99999963099643, 'dc': 70, 'rm': 3.6900357258673466e-07}, 'chrg_1': {'t': 3.1385869565217392, 'p': 7.36}, 'chrg_2': {'t': 5.562517534390779, 'p': 5.775742972106023}}
